In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session 

In [ ]:
! unzip "/kaggle/input/quora-question-pairs/train.csv.zip" -d train

# Eg: here, train is the name-of-directory
# ! unzip "../input/name-of-dataset/train.zip" -d train    


In [ ]:
import numpy as np 
import pandas as pd 
df =pd.read_csv('/kaggle/working/train/train.csv') 

In [ ]:
df.shape

In [ ]:
df.sample(10)

In [ ]:
df.info() 

In [ ]:
df.isnull().sum()  

## Initial EDA

In [ ]:
print(df['is_duplicate'].value_counts())
print(df['is_duplicate'].value_counts()/len(df['id'])*100)
df['is_duplicate'].value_counts().plot(kind='bar')

In [ ]:
df.duplicated().sum()

In [ ]:
qid = pd.Series(df['qid1'].tolist() + df['qid2'].tolist())
print('Number of unique questions',np.unique(qid).shape[0])
x = qid.value_counts()>1
print('Number of questions getting repeated',x[x].shape[0]) 

In [ ]:
# Repeated questions histogram
import matplotlib.pyplot as plt 
plt.hist(qid.value_counts().values,bins=160)
plt.yscale('log')
plt.show() 

## DIRECT MODELLING WITH BOW

In [ ]:
new_df = df.sample(50000) 


In [ ]:
new_df.isnull().sum() 

In [ ]:
new_df.dropna(inplace =True) 

In [ ]:
new_df.isnull().sum()

In [ ]:
ques_df = new_df[['question1','question2']]
ques_df.head() 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=3000)  
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2) 

In [ ]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape 

In [ ]:
temp_df 
 

In [ ]:
temp_df['is_duplicate'] = new_df['is_duplicate'] 

In [ ]:
temp_df.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=1) 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test) 
accuracy_score(y_test,y_pred) 

In [ ]:
# from xgboost import XGBClassifier
# xgb = XGBClassifier()
# xgb.fit(X_train,y_train)
# y_pred = xgb.predict(X_test)
# accuracy_score(y_test,y_pred)


## using tfidf 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 
# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

tfidf = TfidfVectorizer(max_features=3000)  
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2) 

In [ ]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

In [ ]:
temp_df 


In [ ]:
temp_df['is_duplicate'] = new_df['is_duplicate'] 

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=1) 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred) 

## Applying Word2Vec

In [ ]:
import numpy as np
import pandas as pd 
df_3 = pd.read_csv('/kaggle/working/train/train.csv') 

In [ ]:
df_3.dropna(inplace =True )

In [ ]:
df_3.shape

In [ ]:
df_3 =df_3[:50000] 
df_3.shape

In [ ]:
!pip install gensim

In [ ]:
import gensim
import os 
from nltk import sent_tokenize
from gensim.utils import simple_preprocess 

In [ ]:
df_3.head() 

In [ ]:
temp_df = df_3.drop(columns= ['id','qid1','qid2']) 

In [ ]:
temp_df.head() 

In [ ]:
def read_questions(row,column_name):
    return gensim.utils.simple_preprocess(str(row[column_name]).encode('utf-8'))
    
documents = []
for index, row in temp_df.iterrows():
    documents.append(read_questions(row,"question1"))
    if row["is_duplicate"] == 0:
        documents.append(read_questions(row,"question2")) 

In [ ]:
from gensim.models import Word2Vec

w2v_model = Word2Vec(min_count=10,
                     window=3,
                     vector_size=1000,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=4) 

In [ ]:
w2v_model.build_vocab(documents, progress_per=10000)


In [ ]:
w2v_model.train(documents, total_examples=w2v_model.corpus_count, epochs=20, report_delay=1) 

In [ ]:
words = list(w2v_model.wv.key_to_index)


In [ ]:
questions = list(temp_df['question1']) + list(temp_df['question2']) 

In [ ]:
splitted = []
for row in questions: 
    splitted.append([word for word in row.split()]) 

In [ ]:
avg_data = []
for row in splitted:
    vec = np.zeros(1000)
    count = 0
    for word in row:
        try:
            vec = vec+ w2v_model.wv[word]
            count = count+ 1
        except:
            pass
    avg_data.append(vec/count)

In [ ]:
avg_data[0] 

In [ ]:
length = len(avg_data)
half = int(length/2)
first_half = avg_data[:half]
second_half = avg_data[length-half:]

In [ ]:
q1_arry = np.array(first_half)
q2_arry = np.array(second_half)

In [ ]:
q1_arry.shape, q2_arry.shape


In [ ]:
tem_df1 = pd.DataFrame(q1_arry, index= temp_df.index)
tem_df2 = pd.DataFrame(q2_arry, index= temp_df.index)
tem_df = pd.concat([tem_df1, tem_df2], axis=1) 

In [ ]:
temp_df.shape 

In [ ]:
finl_df = pd.concat([tem_df, temp_df], axis=1)


In [ ]:
finl_df.drop(columns=['question1','question2'],inplace =True) 

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain,Xtest,ytrain,ytest = train_test_split(finl_df.iloc[:,0:-1].values,finl_df.iloc[:,-1].values,test_size=0.2,random_state=1) 

In [ ]:
from xgboost import XGBClassifier 
from sklearn.metrics import accuracy_score
xgb = XGBClassifier(n_estimators=200,n_jobs=4, max_depth=30, learning_rate=0.1, subsample = 0.5)
xgb.fit(Xtrain,ytrain)
y_pred = xgb.predict(Xtest) 
accuracy_score(ytest,y_pred) 

# Modelling With Preprocessing and Features

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore') 

In [2]:
import numpy as np
import pandas as pd 
df_3 = pd.read_csv('/kaggle/working/train/train.csv') 

In [3]:
new_df = df_3.sample(30000,random_state=2)


In [4]:
new_df.head()


,id,qid1,qid2,question1,question2,is_duplicate
398782,398782,496695,532029,What is the best marketing automation tool for...,What is the best marketing automation tool for...,1
115086,115086,187729,187730,I am poor but I want to invest. What should I do?,I am quite poor and I want to be very rich. Wh...,0
327711,327711,454161,454162,I am from India and live abroad. I met a guy f...,T.I.E.T to Thapar University to Thapar Univers...,0
367788,367788,498109,491396,Why do so many people in the U.S. hate the sou...,My boyfriend doesnt feel guilty when he hurts ...,0
151235,151235,237843,50930,Consequences of Bhopal gas tragedy?,What was the reason behind the Bhopal gas trag...,0


In [5]:
def preprocess(q):
    
    q = str(q).lower().strip()
    
    # Replace certain special characters with their string equivalents
    q = q.replace('%', ' percent')
    q = q.replace('$', ' dollar ')
    q = q.replace('₹', ' rupee ')
    q = q.replace('€', ' euro ')
    q = q.replace('@', ' at ')
    
    # The pattern '[math]' appears around 900 times in the whole dataset.
    q = q.replace('[math]', '')
    
    # Replacing some numbers with string equivalents (not perfect, can be done better to account for more cases)
    q = q.replace(',000,000,000 ', 'b ')
    q = q.replace(',000,000 ', 'm ')
    q = q.replace(',000 ', 'k ')
    q = re.sub(r'([0-9]+)000000000', r'\1b', q)
    q = re.sub(r'([0-9]+)000000', r'\1m', q)
    q = re.sub(r'([0-9]+)000', r'\1k', q)
    
    # Decontracting words
    # https://en.wikipedia.org/wiki/Wikipedia%3aList_of_English_contractions
    # https://stackoverflow.com/a/19794953
    contractions = { 
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }

    q_decontracted = []

    for word in q.split():
        if word in contractions:
            word = contractions[word]

        q_decontracted.append(word)

    q = ' '.join(q_decontracted)
    q = q.replace("'ve", " have")
    q = q.replace("n't", " not")
    q = q.replace("'re", " are")
    q = q.replace("'ll", " will")
    
    # Removing HTML tags
    q = BeautifulSoup(q)
    q = q.get_text()
    
    # Remove punctuations
    pattern = re.compile('\W')
    q = re.sub(pattern, ' ', q).strip()

    
    return q

In [6]:
preprocess("I've already! wasn't <b>done</b>?")


'i have already  was not done'

In [7]:
new_df['question1'] = new_df['question1'].apply(preprocess)
new_df['question2'] = new_df['question2'].apply(preprocess)

In [8]:
new_df


,id,qid1,qid2,question1,question2,is_duplicate
398782,398782,496695,532029,what is the best marketing automation tool for...,what is the best marketing automation tool for...,1
115086,115086,187729,187730,i am poor but i want to invest what should i do,i am quite poor and i want to be very rich wh...,0
327711,327711,454161,454162,i am from india and live abroad i met a guy f...,t i e t to thapar university to thapar univers...,0
367788,367788,498109,491396,why do so many people in the u s hate the sou...,my boyfriend doesnt feel guilty when he hurts ...,0
151235,151235,237843,50930,consequences of bhopal gas tragedy,what was the reason behind the bhopal gas tragedy,0
...,...,...,...,...,...,...
243932,243932,26193,356455,what are some good web scraping tutorials,what are some good web scraping programs,1
91980,91980,154063,154064,can i apply for internet banking in sbi withou...,i have internet banking kit of sbi but it is n...,0
266955,266955,133017,384210,how much he laundry detergent do you use in a ...,can i use regular dawn dishsoap in my dishwash...,0
71112,71112,122427,122428,what is the best way to understand and learn m...,what are some of the best ways to learn math,1


In [9]:
# ! pip install nltk

In [10]:
import spacy

# Load the spaCy English model
nlp = spacy.load('en_core_web_sm')

# Define a sample text
text = "The quick brown foxes are jumping over the lazy dogs."

# Process the text using spaCy
doc = nlp(text)

# Extract lemmatized tokens
lemmatized_tokens = [token.lemma_ for token in doc]

# Join the lemmatized tokens into a sentence
lemmatized_text = ' '.join(lemmatized_tokens)

# Print the original and lemmatized text
print("Original Text:", text)
print("Lemmatized Text:", lemmatized_text)


Original Text: The quick brown foxes are jumping over the lazy dogs.
Lemmatized Text: the quick brown fox be jump over the lazy dog .


In [11]:
import spacy

# Load the spaCy English model
nlp = spacy.load('en_core_web_sm') 

def lemmatize_words(text):
    doc = nlp(text) 
    lemmatized_tokens = [token.lemma_ for token in doc]
    return ' '.join(lemmatized_tokens) 

new_df["ques1_lemi"] = new_df["question1"].apply(lambda text: lemmatize_words(text))
new_df["ques2_lemi"] = new_df["question2"].apply(lambda text: lemmatize_words(text)) 

In [12]:
new_df 

,id,qid1,qid2,question1,question2,is_duplicate,ques1_lemi,ques2_lemi
398782,398782,496695,532029,what is the best marketing automation tool for...,what is the best marketing automation tool for...,1,what be the good marketing automation tool for...,what be the good marketing automation tool for...
115086,115086,187729,187730,i am poor but i want to invest what should i do,i am quite poor and i want to be very rich wh...,0,I be poor but I want to invest what should I do,I be quite poor and I want to be very rich w...
327711,327711,454161,454162,i am from india and live abroad i met a guy f...,t i e t to thapar university to thapar univers...,0,I be from india and live abroad I meet a guy...,t I e t to thapar university to thapar univers...
367788,367788,498109,491396,why do so many people in the u s hate the sou...,my boyfriend doesnt feel guilty when he hurts ...,0,why do so many people in the u s hate the so...,my boyfriend do not feel guilty when he hurt I...
151235,151235,237843,50930,consequences of bhopal gas tragedy,what was the reason behind the bhopal gas tragedy,0,consequence of bhopal gas tragedy,what be the reason behind the bhopal gas tragedy
...,...,...,...,...,...,...,...,...
243932,243932,26193,356455,what are some good web scraping tutorials,what are some good web scraping programs,1,what be some good web scrape tutorial,what be some good web scrape program
91980,91980,154063,154064,can i apply for internet banking in sbi withou...,i have internet banking kit of sbi but it is n...,0,can I apply for internet banking in sbi withou...,I have internet banking kit of sbi but it be n...
266955,266955,133017,384210,how much he laundry detergent do you use in a ...,can i use regular dawn dishsoap in my dishwash...,0,how much he laundry detergent do you use in a ...,can I use regular dawn dishsoap in my dishwash...
71112,71112,122427,122428,what is the best way to understand and learn m...,what are some of the best ways to learn math,1,what be the good way to understand and learn math,what be some of the good way to learn math


In [13]:
new_df = new_df.drop(['question1','question2'], axis=1)


In [14]:
new_df

,id,qid1,qid2,is_duplicate,ques1_lemi,ques2_lemi
398782,398782,496695,532029,1,what be the good marketing automation tool for...,what be the good marketing automation tool for...
115086,115086,187729,187730,0,I be poor but I want to invest what should I do,I be quite poor and I want to be very rich w...
327711,327711,454161,454162,0,I be from india and live abroad I meet a guy...,t I e t to thapar university to thapar univers...
367788,367788,498109,491396,0,why do so many people in the u s hate the so...,my boyfriend do not feel guilty when he hurt I...
151235,151235,237843,50930,0,consequence of bhopal gas tragedy,what be the reason behind the bhopal gas tragedy
...,...,...,...,...,...,...
243932,243932,26193,356455,1,what be some good web scrape tutorial,what be some good web scrape program
91980,91980,154063,154064,0,can I apply for internet banking in sbi withou...,I have internet banking kit of sbi but it be n...
266955,266955,133017,384210,0,how much he laundry detergent do you use in a ...,can I use regular dawn dishsoap in my dishwash...
71112,71112,122427,122428,1,what be the good way to understand and learn math,what be some of the good way to learn math


In [15]:
new_df['q1_len'] = new_df['ques1_lemi'].str.len() 
new_df['q2_len'] = new_df['ques2_lemi'].str.len() 

In [16]:
new_df['q1_num_words'] = new_df['ques1_lemi'].apply(lambda row: len(row.split(" ")))
new_df['q2_num_words'] = new_df['ques2_lemi'].apply(lambda row: len(row.split(" ")))
new_df.head()  

,id,qid1,qid2,is_duplicate,ques1_lemi,ques2_lemi,q1_len,q2_len,q1_num_words,q2_num_words
398782,398782,496695,532029,1,what be the good marketing automation tool for...,what be the good marketing automation tool for...,73,74,13,13
115086,115086,187729,187730,0,I be poor but I want to invest what should I do,I be quite poor and I want to be very rich w...,49,57,14,17
327711,327711,454161,454162,0,I be from india and live abroad I meet a guy...,t I e t to thapar university to thapar univers...,106,120,30,22
367788,367788,498109,491396,0,why do so many people in the u s hate the so...,my boyfriend do not feel guilty when he hurt I...,58,135,15,35
151235,151235,237843,50930,0,consequence of bhopal gas tragedy,what be the reason behind the bhopal gas tragedy,33,48,5,9


In [17]:
def common_words(row):
    w1 = set(map(lambda word: word.lower().strip(), row['ques1_lemi'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['ques2_lemi'].split(" ")))    
    return len(w1 & w2) 

In [18]:
new_df['word_common'] = new_df.apply(common_words, axis=1)
new_df.head()

,id,qid1,qid2,is_duplicate,ques1_lemi,ques2_lemi,q1_len,q2_len,q1_num_words,q2_num_words,word_common
398782,398782,496695,532029,1,what be the good marketing automation tool for...,what be the good marketing automation tool for...,73,74,13,13,12
115086,115086,187729,187730,0,I be poor but I want to invest what should I do,I be quite poor and I want to be very rich w...,49,57,14,17,8
327711,327711,454161,454162,0,I be from india and live abroad I meet a guy...,t I e t to thapar university to thapar univers...,106,120,30,22,4
367788,367788,498109,491396,0,why do so many people in the u s hate the so...,my boyfriend do not feel guilty when he hurt I...,58,135,15,35,2
151235,151235,237843,50930,0,consequence of bhopal gas tragedy,what be the reason behind the bhopal gas tragedy,33,48,5,9,3


In [19]:
def total_words(row):
    w1 = set(map(lambda word: word.lower().strip(), row['ques1_lemi'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['ques2_lemi'].split(" ")))    
    return (len(w1) + len(w2))

In [20]:
new_df['word_total'] = new_df.apply(total_words, axis=1)
new_df.head() 

,id,qid1,qid2,is_duplicate,ques1_lemi,ques2_lemi,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total
398782,398782,496695,532029,1,what be the good marketing automation tool for...,what be the good marketing automation tool for...,73,74,13,13,12,26
115086,115086,187729,187730,0,I be poor but I want to invest what should I do,I be quite poor and I want to be very rich w...,49,57,14,17,8,23
327711,327711,454161,454162,0,I be from india and live abroad I meet a guy...,t I e t to thapar university to thapar univers...,106,120,30,22,4,38
367788,367788,498109,491396,0,why do so many people in the u s hate the so...,my boyfriend do not feel guilty when he hurt I...,58,135,15,35,2,34
151235,151235,237843,50930,0,consequence of bhopal gas tragedy,what be the reason behind the bhopal gas tragedy,33,48,5,9,3,13


In [21]:
new_df['word_share'] = round(new_df['word_common']/new_df['word_total'],2)
new_df.head() 

,id,qid1,qid2,is_duplicate,ques1_lemi,ques2_lemi,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total,word_share
398782,398782,496695,532029,1,what be the good marketing automation tool for...,what be the good marketing automation tool for...,73,74,13,13,12,26,0.46
115086,115086,187729,187730,0,I be poor but I want to invest what should I do,I be quite poor and I want to be very rich w...,49,57,14,17,8,23,0.35
327711,327711,454161,454162,0,I be from india and live abroad I meet a guy...,t I e t to thapar university to thapar univers...,106,120,30,22,4,38,0.11
367788,367788,498109,491396,0,why do so many people in the u s hate the so...,my boyfriend do not feel guilty when he hurt I...,58,135,15,35,2,34,0.06
151235,151235,237843,50930,0,consequence of bhopal gas tragedy,what be the reason behind the bhopal gas tragedy,33,48,5,9,3,13,0.23


In [22]:
# Advanced Features
from nltk.corpus import stopwords

def fetch_token_features(row):
    
    q1 = row['ques1_lemi']
    q2 = row['ques2_lemi']
    
    SAFE_DIV = 0.0001 

    STOP_WORDS = stopwords.words("english")
    
    token_features = [0.0]*8
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split() 
    q2_tokens = q2.split()
    
    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return token_features

    # Get the non-stopwords in Questions
    q1_words = set([word for word in q1_tokens if word not in STOP_WORDS])
    q2_words = set([word for word in q2_tokens if word not in STOP_WORDS])
    
    #Get the stopwords in Questions
    q1_stops = set([word for word in q1_tokens if word in STOP_WORDS])
    q2_stops = set([word for word in q2_tokens if word in STOP_WORDS])
    
    # Get the common non-stopwords from Question pair
    common_word_count = len(q1_words.intersection(q2_words))
    
    # Get the common stopwords from Question pair
    common_stop_count = len(q1_stops.intersection(q2_stops))
    
    # Get the common Tokens from Question pair
    common_token_count = len(set(q1_tokens).intersection(set(q2_tokens)))
    
    
    token_features[0] = common_word_count / (min(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[1] = common_word_count / (max(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[2] = common_stop_count / (min(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[3] = common_stop_count / (max(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[4] = common_token_count / (min(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    token_features[5] = common_token_count / (max(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    
    # Last word of both question is same or not
    token_features[6] = int(q1_tokens[-1] == q2_tokens[-1])
    
    # First word of both question is same or not
    token_features[7] = int(q1_tokens[0] == q2_tokens[0])
    
    return token_features

In [23]:
token_features = new_df.apply(fetch_token_features, axis=1)

new_df["cwc_min"]       = list(map(lambda x: x[0], token_features))
new_df["cwc_max"]       = list(map(lambda x: x[1], token_features))
new_df["csc_min"]       = list(map(lambda x: x[2], token_features))
new_df["csc_max"]       = list(map(lambda x: x[3], token_features))
new_df["ctc_min"]       = list(map(lambda x: x[4], token_features))
new_df["ctc_max"]       = list(map(lambda x: x[5], token_features))
new_df["last_word_eq"]  = list(map(lambda x: x[6], token_features))
new_df["first_word_eq"] = list(map(lambda x: x[7], token_features))

In [24]:
new_df.head() 

,id,qid1,qid2,is_duplicate,ques1_lemi,ques2_lemi,q1_len,q2_len,q1_num_words,q2_num_words,...,word_total,word_share,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq
398782,398782,496695,532029,1,what be the good marketing automation tool for...,what be the good marketing automation tool for...,73,74,13,13,...,26,0.46,0.874989,0.874989,0.999980,0.999980,0.923070,0.923070,1.0,1.0
115086,115086,187729,187730,0,I be poor but I want to invest what should I do,I be quite poor and I want to be very rich w...,49,57,14,17,...,23,0.35,0.749981,0.599988,0.666656,0.666656,0.583328,0.466664,1.0,1.0
327711,327711,454161,454162,0,I be from india and live abroad I meet a guy...,t I e t to thapar university to thapar univers...,106,120,30,22,...,38,0.11,0.099999,0.099999,0.333328,0.199998,0.149999,0.115384,0.0,0.0
367788,367788,498109,491396,0,why do so many people in the u s hate the so...,my boyfriend do not feel guilty when he hurt I...,58,135,15,35,...,34,0.06,0.000000,0.000000,0.166664,0.124998,0.076922,0.032258,0.0,0.0
151235,151235,237843,50930,0,consequence of bhopal gas tragedy,what be the reason behind the bhopal gas tragedy,33,48,5,9,...,13,0.23,0.749981,0.599988,0.000000,0.000000,0.599988,0.333330,1.0,0.0


In [25]:
! pip install distance 

In [26]:
import distance

def fetch_length_features(row):
    
    q1 = row['ques1_lemi']
    q2 = row['ques2_lemi']
    
    length_features = [0.0]*3
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()
    
    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return length_features
    
    # Absolute length features
    length_features[0] = abs(len(q1_tokens) - len(q2_tokens))
    
    #Average Token Length of both Questions
    length_features[1] = (len(q1_tokens) + len(q2_tokens))/2
    
    strs = list(distance.lcsubstrings(q1, q2))
    length_features[2] = len(strs[0]) / (min(len(q1), len(q2)) + 1)
    
    return length_features

In [27]:
length_features = new_df.apply(fetch_length_features, axis=1)

new_df['abs_len_diff'] = list(map(lambda x: x[0], length_features))
new_df['mean_len'] = list(map(lambda x: x[1], length_features))
new_df['longest_substr_ratio'] = list(map(lambda x: x[2], length_features)) 

In [28]:
new_df.head()


,id,qid1,qid2,is_duplicate,ques1_lemi,ques2_lemi,q1_len,q2_len,q1_num_words,q2_num_words,...,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,longest_substr_ratio
398782,398782,496695,532029,1,what be the good marketing automation tool for...,what be the good marketing automation tool for...,73,74,13,13,...,0.874989,0.999980,0.999980,0.923070,0.923070,1.0,1.0,0.0,13.0,0.878378
115086,115086,187729,187730,0,I be poor but I want to invest what should I do,I be quite poor and I want to be very rich w...,49,57,14,17,...,0.599988,0.666656,0.666656,0.583328,0.466664,1.0,1.0,3.0,13.5,0.220000
327711,327711,454161,454162,0,I be from india and live abroad I meet a guy...,t I e t to thapar university to thapar univers...,106,120,30,22,...,0.099999,0.333328,0.199998,0.149999,0.115384,0.0,0.0,6.0,23.0,0.046729
367788,367788,498109,491396,0,why do so many people in the u s hate the so...,my boyfriend do not feel guilty when he hurt I...,58,135,15,35,...,0.000000,0.166664,0.124998,0.076922,0.032258,0.0,0.0,18.0,22.0,0.067797
151235,151235,237843,50930,0,consequence of bhopal gas tragedy,what be the reason behind the bhopal gas tragedy,33,48,5,9,...,0.599988,0.000000,0.000000,0.599988,0.333330,1.0,0.0,4.0,7.0,0.558824


In [29]:
# Fuzzy Features
from fuzzywuzzy import fuzz

def fetch_fuzzy_features(row):
    
    q1 = row['ques1_lemi']
    q2 = row['ques2_lemi']
    
    fuzzy_features = [0.0]*4
    
    # fuzz_ratio
    fuzzy_features[0] = fuzz.QRatio(q1, q2)

    # fuzz_partial_ratio
    fuzzy_features[1] = fuzz.partial_ratio(q1, q2)

    # token_sort_ratio
    fuzzy_features[2] = fuzz.token_sort_ratio(q1, q2)

    # token_set_ratio
    fuzzy_features[3] = fuzz.token_set_ratio(q1, q2)

    return fuzzy_features 

In [30]:
fuzzy_features = new_df.apply(fetch_fuzzy_features, axis=1)

# Creating new feature columns for fuzzy features
new_df['fuzz_ratio'] = list(map(lambda x: x[0], fuzzy_features))
new_df['fuzz_partial_ratio'] = list(map(lambda x: x[1], fuzzy_features))
new_df['token_sort_ratio'] = list(map(lambda x: x[2], fuzzy_features))
new_df['token_set_ratio'] = list(map(lambda x: x[3], fuzzy_features)) 

In [31]:
print(new_df.shape)
new_df.head() 

(30000, 28)


,id,qid1,qid2,is_duplicate,ques1_lemi,ques2_lemi,q1_len,q2_len,q1_num_words,q2_num_words,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,longest_substr_ratio,fuzz_ratio,fuzz_partial_ratio,token_sort_ratio,token_set_ratio
398782,398782,496695,532029,1,what be the good marketing automation tool for...,what be the good marketing automation tool for...,73,74,13,13,...,0.923070,1.0,1.0,0.0,13.0,0.878378,99,99,99,99
115086,115086,187729,187730,0,I be poor but I want to invest what should I do,I be quite poor and I want to be very rich w...,49,57,14,17,...,0.466664,1.0,1.0,3.0,13.5,0.220000,70,69,65,74
327711,327711,454161,454162,0,I be from india and live abroad I meet a guy...,t I e t to thapar university to thapar univers...,106,120,30,22,...,0.115384,0.0,0.0,6.0,23.0,0.046729,29,36,33,42
367788,367788,498109,491396,0,why do so many people in the u s hate the so...,my boyfriend do not feel guilty when he hurt I...,58,135,15,35,...,0.032258,0.0,0.0,18.0,22.0,0.067797,32,47,18,28
151235,151235,237843,50930,0,consequence of bhopal gas tragedy,what be the reason behind the bhopal gas tragedy,33,48,5,9,...,0.333330,1.0,0.0,4.0,7.0,0.558824,57,71,49,71


In [32]:
# sns.pairplot(new_df[['ctc_min', 'cwc_min', 'csc_min', 'is_duplicate']],hue='is_duplicate')


In [33]:
# sns.pairplot(new_df[['ctc_max', 'cwc_max', 'csc_max', 'is_duplicate']],hue='is_duplicate') 


In [34]:
# sns.pairplot(new_df[['last_word_eq', 'first_word_eq', 'is_duplicate']],hue='is_duplicate') 


In [35]:
# sns.pairplot(new_df[['mean_len', 'abs_len_diff','longest_substr_ratio', 'is_duplicate']],hue='is_duplicate')


In [36]:
# sns.pairplot(new_df[['fuzz_ratio', 'fuzz_partial_ratio','token_sort_ratio','token_set_ratio', 'is_duplicate']],hue='is_duplicate')


In [37]:
# Using TSNE for Dimentionality reduction for 15 Features(Generated after cleaning the data) to 3 dimention

from sklearn.preprocessing import MinMaxScaler

X = MinMaxScaler().fit_transform(new_df[['cwc_min', 'cwc_max', 'csc_min', 'csc_max' , 'ctc_min' , 'ctc_max' , 'last_word_eq', 'first_word_eq' , 'abs_len_diff' , 'mean_len' , 'token_set_ratio' , 'token_sort_ratio' ,  'fuzz_ratio' , 'fuzz_partial_ratio' , 'longest_substr_ratio']])
y = new_df['is_duplicate'].values 

In [38]:
# from sklearn.manifold import TSNE

# tsne2d = TSNE(
#     n_components=2,
#     init='random', # pca
#     random_state=101,
#     method='barnes_hut',
#     n_iter=1000,
#     verbose=2,
#     angle=0.5
# ).fit_transform(X)

In [39]:
# x_df = pd.DataFrame({'x':tsne2d[:,0], 'y':tsne2d[:,1] ,'label':y})

# # draw the plot in appropriate place in the grid
# sns.lmplot(data=x_df, x='x', y='y', hue='label', fit_reg=False, size=8,palette="Set1",markers=['s','o'])

In [40]:
# tsne3d = TSNE(
#     n_components=3,
#     init='random', # pca
#     random_state=101,
#     method='barnes_hut',
#     n_iter=1000,
#     verbose=2,
#     angle=0.5
# ).fit_transform(X)

In [41]:
# import plotly.graph_objs as go
# import plotly.tools as tls
# import plotly.offline as py
# py.init_notebook_mode(connected=True)

# trace1 = go.Scatter3d(
#     x=tsne3d[:,0],
#     y=tsne3d[:,1],
#     z=tsne3d[:,2],
#     mode='markers',
#     marker=dict(
#         sizemode='diameter',
#         color = y,
#         colorscale = 'Portland',
#         colorbar = dict(title = 'duplicate'),
#         line=dict(color='rgb(255, 255, 255)'),
#         opacity=0.75
#     )
# )

# data=[trace1]
# layout=dict(height=800, width=800, title='3d embedding with engineered features')
# fig=dict(data=data, layout=layout)
# py.iplot(fig, filename='3DBubble')

In [42]:
ques_df = new_df[['ques1_lemi','ques2_lemi']]
ques_df.head() 

,ques1_lemi,ques2_lemi
398782,what be the good marketing automation tool for...,what be the good marketing automation tool for...
115086,I be poor but I want to invest what should I do,I be quite poor and I want to be very rich w...
327711,I be from india and live abroad I meet a guy...,t I e t to thapar university to thapar univers...
367788,why do so many people in the u s hate the so...,my boyfriend do not feel guilty when he hurt I...
151235,consequence of bhopal gas tragedy,what be the reason behind the bhopal gas tragedy


In [43]:
final_df = new_df.drop(columns=['id','qid1','qid2','ques1_lemi','ques2_lemi'])
print(final_df.shape)
final_df.head()

(30000, 23)


,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total,word_share,cwc_min,cwc_max,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,longest_substr_ratio,fuzz_ratio,fuzz_partial_ratio,token_sort_ratio,token_set_ratio
398782,1,73,74,13,13,12,26,0.46,0.874989,0.874989,...,0.923070,1.0,1.0,0.0,13.0,0.878378,99,99,99,99
115086,0,49,57,14,17,8,23,0.35,0.749981,0.599988,...,0.466664,1.0,1.0,3.0,13.5,0.220000,70,69,65,74
327711,0,106,120,30,22,4,38,0.11,0.099999,0.099999,...,0.115384,0.0,0.0,6.0,23.0,0.046729,29,36,33,42
367788,0,58,135,15,35,2,34,0.06,0.000000,0.000000,...,0.032258,0.0,0.0,18.0,22.0,0.067797,32,47,18,28
151235,0,33,48,5,9,3,13,0.23,0.749981,0.599988,...,0.333330,1.0,0.0,4.0,7.0,0.558824,57,71,49,71


In [44]:
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(ques_df['ques1_lemi']) + list(ques_df['ques2_lemi'])

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [45]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape 

(30000, 6000)

In [46]:
final_df = pd.concat([final_df, temp_df], axis=1)
print(final_df.shape)
final_df

(30000, 6023)


,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total,word_share,cwc_min,cwc_max,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
398782,1,73,74,13,13,12,26,0.46,0.874989,0.874989,...,0,0,0,0,0,0,0,0,0,0
115086,0,49,57,14,17,8,23,0.35,0.749981,0.599988,...,0,0,0,0,0,0,0,0,0,0
327711,0,106,120,30,22,4,38,0.11,0.099999,0.099999,...,0,0,0,0,0,0,0,0,0,0
367788,0,58,135,15,35,2,34,0.06,0.000000,0.000000,...,1,0,0,0,0,0,0,0,0,0
151235,0,33,48,5,9,3,13,0.23,0.749981,0.599988,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243932,1,37,36,7,7,6,14,0.43,0.749981,0.749981,...,0,0,0,0,0,0,0,0,0,0
91980,0,64,59,12,15,4,26,0.15,0.666656,0.499994,...,0,0,0,0,0,0,0,0,0,0
266955,0,72,99,14,21,6,31,0.19,0.499994,0.444440,...,0,0,0,0,0,0,0,0,0,0
71112,1,49,42,10,10,8,20,0.40,0.999975,0.799984,...,0,0,0,0,0,0,0,0,0,0


In [47]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(final_df.iloc[:,1:].values,final_df.iloc[:,0].values,test_size=0.2,random_state=1)

In [60]:
X_test.shape

(6000, 6022)

In [48]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)



y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred) 

0.786

In [49]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred1 = xgb.predict(X_test)
accuracy_score(y_test,y_pred1)

0.7836666666666666

In [50]:
from sklearn.metrics import confusion_matrix


In [51]:
confusion_matrix(y_test,y_pred)


array([[3331,  481],
       [ 803, 1385]])

In [52]:
# for xgboost model
confusion_matrix(y_test,y_pred1)

array([[3253,  559],
       [ 739, 1449]])

In [53]:
def test_common_words(q1,q2):
    w1 = set(map(lambda word: word.lower().strip(), q1.split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), q2.split(" ")))    
    return len(w1 & w2)

In [54]:
def test_total_words(q1,q2):
    w1 = set(map(lambda word: word.lower().strip(), q1.split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), q2.split(" ")))    
    return (len(w1) + len(w2))

In [55]:
def test_fetch_token_features(q1,q2):
    
    SAFE_DIV = 0.0001 

    STOP_WORDS = stopwords.words("english")
    
    token_features = [0.0]*8
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()
    
    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return token_features

    # Get the non-stopwords in Questions
    q1_words = set([word for word in q1_tokens if word not in STOP_WORDS])
    q2_words = set([word for word in q2_tokens if word not in STOP_WORDS])
    
    #Get the stopwords in Questions
    q1_stops = set([word for word in q1_tokens if word in STOP_WORDS])
    q2_stops = set([word for word in q2_tokens if word in STOP_WORDS])
    
    # Get the common non-stopwords from Question pair
    common_word_count = len(q1_words.intersection(q2_words))
    
    # Get the common stopwords from Question pair
    common_stop_count = len(q1_stops.intersection(q2_stops))
    
    # Get the common Tokens from Question pair
    common_token_count = len(set(q1_tokens).intersection(set(q2_tokens)))
    
    
    token_features[0] = common_word_count / (min(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[1] = common_word_count / (max(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[2] = common_stop_count / (min(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[3] = common_stop_count / (max(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[4] = common_token_count / (min(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    token_features[5] = common_token_count / (max(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    
    # Last word of both question is same or not
    token_features[6] = int(q1_tokens[-1] == q2_tokens[-1])
    
    # First word of both question is same or not
    token_features[7] = int(q1_tokens[0] == q2_tokens[0])
    
    return token_features

In [56]:
def test_fetch_length_features(q1,q2):
    
    length_features = [0.0]*3
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()
    
    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return length_features
    
    # Absolute length features
    length_features[0] = abs(len(q1_tokens) - len(q2_tokens))
    
    #Average Token Length of both Questions
    length_features[1] = (len(q1_tokens) + len(q2_tokens))/2
    
    strs = list(distance.lcsubstrings(q1, q2))
    length_features[2] = len(strs[0]) / (min(len(q1), len(q2)) + 1)
    
    return length_features

In [57]:
def test_fetch_fuzzy_features(q1,q2):
    
    fuzzy_features = [0.0]*4
    
    # fuzz_ratio
    fuzzy_features[0] = fuzz.QRatio(q1, q2)

    # fuzz_partial_ratio
    fuzzy_features[1] = fuzz.partial_ratio(q1, q2)

    # token_sort_ratio
    fuzzy_features[2] = fuzz.token_sort_ratio(q1, q2)

    # token_set_ratio
    fuzzy_features[3] = fuzz.token_set_ratio(q1, q2)

    return fuzzy_features

In [58]:
def query_point_creator(q1,q2):
    
    input_query = []
    
    # preprocess
    q1 = preprocess(q1)
    q2 = preprocess(q2)
    
    # fetch basic features
    input_query.append(len(q1))
    input_query.append(len(q2))
    
    input_query.append(len(q1.split(" ")))
    input_query.append(len(q2.split(" ")))
    
    input_query.append(test_common_words(q1,q2))
    input_query.append(test_total_words(q1,q2))
    input_query.append(round(test_common_words(q1,q2)/test_total_words(q1,q2),2))
    
    # fetch token features
    token_features = test_fetch_token_features(q1,q2)
    input_query.extend(token_features)
    
    # fetch length based features
    length_features = test_fetch_length_features(q1,q2)
    input_query.extend(length_features)
    
    # fetch fuzzy features
    fuzzy_features = test_fetch_fuzzy_features(q1,q2)
    input_query.extend(fuzzy_features)
    
    # bow feature for q1
    q1_bow = cv.transform([q1]).toarray()
    
    # bow feature for q2
    q2_bow = cv.transform([q2]).toarray()
    
    
    
    return np.hstack((np.array(input_query).reshape(1,22),q1_bow,q2_bow))

In [59]:
q1 = 'Where is the capital of India?'
q2 = 'What is the current capital of Pakistan?'
q3 = 'Which city serves as the capital of India?'
q4 = 'What is the business capital of India?'

In [61]:
rf.predict(query_point_creator(q1,q4))


array([1])

In [62]:
cv


CountVectorizer(max_features=3000)

In [63]:
import pickle

pickle.dump(rf,open('model.pkl','wb'))
pickle.dump(cv,open('cv.pkl','wb')) 